<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Working-with-Text" data-toc-modified-id="Working-with-Text-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Working with Text</a></span><ul class="toc-item"><li><span><a href="#String-split" data-toc-modified-id="String-split-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>String split</a></span></li><li><span><a href="#String-replacement" data-toc-modified-id="String-replacement-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>String replacement</a></span></li><li><span><a href="#Substrings" data-toc-modified-id="Substrings-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Substrings</a></span></li><li><span><a href="#Pattern-matching" data-toc-modified-id="Pattern-matching-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Pattern matching</a></span></li></ul></li><li><span><a href="#Time-Series" data-toc-modified-id="Time-Series-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Time Series</a></span><ul class="toc-item"><li><span><a href="#Timestamps-and-Time-Spans" data-toc-modified-id="Timestamps-and-Time-Spans-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Timestamps and Time Spans</a></span></li><li><span><a href="#Converting-to-Timestamps" data-toc-modified-id="Converting-to-Timestamps-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Converting to Timestamps</a></span></li><li><span><a href="#Time/data-components" data-toc-modified-id="Time/data-components-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Time/data components</a></span></li></ul></li></ul></div>

# Setup

In [1]:
import pandas as pd
import numpy as np

# Working with Text

## String split

In [2]:
D = {
    "id": [0, 1, 2],
    "s": ["a_b", "B_d", "g_H"]
}
DF = pd.DataFrame(D)

In [3]:
DF

,id,s
0,0,a_b
1,1,B_d
2,2,g_H


In [9]:
# substring occurring before the split
DF.loc[:, "s0"] = DF["s"].str.split("_").str.get(0)

In [10]:
# substring occurring after the split
DF.loc[:, "s1"] = DF["s"].str.split("_").str.get(1)

In [15]:
DF

,s,s_1,s0,s1
id,,,,
0,a_b,a,a,b
1,B_d,B,B,d
2,g_H,g,g,H


In [12]:
# expand option
DF.set_index("id", inplace=True)
DF0 = DF["s"].str.split("_", expand=True)
DF0

,0,1
id,,
0,a,b
1,B,d
2,g,H


In [13]:
# change var names
DF0.rename(columns={0: "s_0", 1: "s_1"}, inplace=True)
DF0

,s_0,s_1
id,,
0,a,b
1,B,d
2,g,H


In [16]:
# remove some junk in the DF
DF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 0 to 2
Data columns (total 4 columns):
s      3 non-null object
s_1    3 non-null object
s0     3 non-null object
s1     3 non-null object
dtypes: object(4)
memory usage: 200.0+ bytes


In [17]:
DF.drop(["s_1", "s0", "s1"], axis=1, inplace=True)
DF

,s
id,
0,a_b
1,B_d
2,g_H


In [18]:
# merge DF and DF0
DF = pd.merge(DF, DF0, left_index=True, right_index=True)
DF

,s,s_0,s_1
id,,,
0,a_b,a,b
1,B_d,B,d
2,g_H,g,H


## String replacement

In [19]:
DF.loc[:,'s_2'] = DF["s"].str.replace("_", "--")

In [20]:
DF

,s,s_0,s_1,s_2
id,,,,
0,a_b,a,b,a--b
1,B_d,B,d,B--d
2,g_H,g,H,g--H


## Substrings

Tutorial on regular expressions: 
https://www.w3schools.com/jsref/jsref_obj_regexp.asp

In [31]:
DF["s"].str.extract("(?P<letter>[A-Z])")

,letter
id,
0,NaN
1,B
2,H


## Pattern matching

In [32]:
DF.loc[:, "match"] = DF["s"].str.contains("a")

In [33]:
DF

,s,s_0,s_1,s_2,s_3,match
id,,,,,,
0,a_b,a,b,a--b,NaN,True
1,B_d,B,d,B--d,NaN,False
2,g_H,g,H,g--H,NaN,False


# Time Series

## Timestamps and Time Spans

In [35]:
# timestamps
pd.Timestamp(2014, 3, 11)

Timestamp('2014-03-11 00:00:00')

In [36]:
# time spans
pd.Period("2014-03", freq="M")

Period('2014-03', 'M')

## Converting to Timestamps

In [42]:
DF = pd.read_csv("reviews_Grocery_and_Gourmet_Food_5.csv")

In [43]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151254 entries, 0 to 151253
Data columns (total 9 columns):
asin              151254 non-null object
helpful           151254 non-null object
overall           151254 non-null float64
reviewText        151232 non-null object
reviewTime        151254 non-null object
reviewerID        151254 non-null object
reviewerName      149761 non-null object
summary           151254 non-null object
unixReviewTime    151254 non-null int64
dtypes: float64(1), int64(1), object(7)
memory usage: 10.4+ MB


In [44]:
DF.head()["reviewTime"]

0     06 1, 2013
1    05 19, 2014
2     10 8, 2013
3    05 20, 2013
4    05 26, 2013
Name: reviewTime, dtype: object

In [45]:
DF.loc[:, "timestamp"] = pd.to_datetime(
    DF["reviewTime"],
    format="%m %d, %Y"
)

In [47]:
DF.head()[["reviewTime", "timestamp"]]

,reviewTime,timestamp
0,"06 1, 2013",2013-06-01
1,"05 19, 2014",2014-05-19
2,"10 8, 2013",2013-10-08
3,"05 20, 2013",2013-05-20
4,"05 26, 2013",2013-05-26


In [48]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151254 entries, 0 to 151253
Data columns (total 10 columns):
asin              151254 non-null object
helpful           151254 non-null object
overall           151254 non-null float64
reviewText        151232 non-null object
reviewTime        151254 non-null object
reviewerID        151254 non-null object
reviewerName      149761 non-null object
summary           151254 non-null object
unixReviewTime    151254 non-null int64
timestamp         151254 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(7)
memory usage: 11.5+ MB


In [49]:
DF.set_index("timestamp", inplace=True)

In [50]:
DF.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
timestamp,,,,,,,,,
2013-06-01,616719923X,"[0, 0]",4.0,Just another flavor of Kit Kat but the taste i...,"06 1, 2013",A1VEELTKS8NLZB,Amazon Customer,Good Taste,1370044800
2014-05-19,616719923X,"[0, 1]",3.0,I bought this on impulse and it comes from Jap...,"05 19, 2014",A14R9XMZVJ6INB,amf0001,"3.5 stars, sadly not as wonderful as I had hoped",1400457600
2013-10-08,616719923X,"[3, 4]",4.0,Really good. Great gift for any fan of green t...,"10 8, 2013",A27IQHDZFQFNGG,Caitlin,Yum!,1381190400
2013-05-20,616719923X,"[0, 0]",5.0,"I had never had it before, was curious to see ...","05 20, 2013",A31QY5TASILE89,DebraDownSth,Unexpected flavor meld,1369008000
2013-05-26,616719923X,"[1, 2]",4.0,I've been looking forward to trying these afte...,"05 26, 2013",A2LWK003FFMCI5,Diana X.,"Not a very strong tea flavor, but still yummy ...",1369526400


In [53]:
DF["2013-05-01":"2013-05-02"].head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
timestamp,,,,,,,,,
2013-05-02,B0002E2GQU,"[0, 0]",5.0,Lavazza produces wonderful coffee and this is ...,"05 2, 2013",A1N7L4DRG7XS2Q,Leslie Heintzberger,This deserves 10 stars.,1367452800
2013-05-01,B00032G1S0,"[4, 4]",5.0,Finally I found it! THE Tuscan milk....OMG! Ev...,"05 1, 2013",A2GA64WZK8PKBC,Cat Feet,Great as a facial wash AND bath!,1367366400
2013-05-02,B0004MTMD0,"[1, 1]",3.0,This cocoa chile blend was nice and spicy but ...,"05 2, 2013",A321CGNKNMYU9N,"J. Conrad ""pandorado""",Spicy but not chocolatey enough,1367452800
2013-05-01,B0005XOVY8,"[0, 0]",5.0,I start my day with a cup of coffee with sugar...,"05 1, 2013",A25HBMBIBGXCQI,"L. Martin ""RenoGirl""",Great Way to Start Your Day!,1367366400
2013-05-01,B0005Z8NCM,"[0, 2]",1.0,The taste of this stuff is sooooo unnatural; I...,"05 1, 2013",A34V43R4VN7OFD,D. Lopez,No!,1367366400


## Time/data components

In [54]:
DF.reset_index(inplace=True)

In [55]:
DF.head()

,timestamp,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,2013-06-01,616719923X,"[0, 0]",4.0,Just another flavor of Kit Kat but the taste i...,"06 1, 2013",A1VEELTKS8NLZB,Amazon Customer,Good Taste,1370044800
1,2014-05-19,616719923X,"[0, 1]",3.0,I bought this on impulse and it comes from Jap...,"05 19, 2014",A14R9XMZVJ6INB,amf0001,"3.5 stars, sadly not as wonderful as I had hoped",1400457600
2,2013-10-08,616719923X,"[3, 4]",4.0,Really good. Great gift for any fan of green t...,"10 8, 2013",A27IQHDZFQFNGG,Caitlin,Yum!,1381190400
3,2013-05-20,616719923X,"[0, 0]",5.0,"I had never had it before, was curious to see ...","05 20, 2013",A31QY5TASILE89,DebraDownSth,Unexpected flavor meld,1369008000
4,2013-05-26,616719923X,"[1, 2]",4.0,I've been looking forward to trying these afte...,"05 26, 2013",A2LWK003FFMCI5,Diana X.,"Not a very strong tea flavor, but still yummy ...",1369526400


In [56]:
# .dt accessor of Pandas
DF.loc[:, "review_year"] = DF["timestamp"].dt.year

In [57]:
DF.head().T

,0,1,2,3,4
timestamp,2013-06-01 00:00:00,2014-05-19 00:00:00,2013-10-08 00:00:00,2013-05-20 00:00:00,2013-05-26 00:00:00
asin,616719923X,616719923X,616719923X,616719923X,616719923X
helpful,"[0, 0]","[0, 1]","[3, 4]","[0, 0]","[1, 2]"
overall,4,3,4,5,4
reviewText,Just another flavor of Kit Kat but the taste i...,I bought this on impulse and it comes from Jap...,Really good. Great gift for any fan of green t...,"I had never had it before, was curious to see ...",I've been looking forward to trying these afte...
reviewTime,"06 1, 2013","05 19, 2014","10 8, 2013","05 20, 2013","05 26, 2013"
reviewerID,A1VEELTKS8NLZB,A14R9XMZVJ6INB,A27IQHDZFQFNGG,A31QY5TASILE89,A2LWK003FFMCI5
reviewerName,Amazon Customer,amf0001,Caitlin,DebraDownSth,Diana X.
summary,Good Taste,"3.5 stars, sadly not as wonderful as I had hoped",Yum!,Unexpected flavor meld,"Not a very strong tea flavor, but still yummy ..."
unixReviewTime,1370044800,1400457600,1381190400,1369008000,1369526400


In [58]:
DF.loc[:, "review_month"] = DF["timestamp"].dt.month
DF.loc[:, "review_day"] = DF["timestamp"].dt.day

In [59]:
DF.head().T

,0,1,2,3,4
timestamp,2013-06-01 00:00:00,2014-05-19 00:00:00,2013-10-08 00:00:00,2013-05-20 00:00:00,2013-05-26 00:00:00
asin,616719923X,616719923X,616719923X,616719923X,616719923X
helpful,"[0, 0]","[0, 1]","[3, 4]","[0, 0]","[1, 2]"
overall,4,3,4,5,4
reviewText,Just another flavor of Kit Kat but the taste i...,I bought this on impulse and it comes from Jap...,Really good. Great gift for any fan of green t...,"I had never had it before, was curious to see ...",I've been looking forward to trying these afte...
reviewTime,"06 1, 2013","05 19, 2014","10 8, 2013","05 20, 2013","05 26, 2013"
reviewerID,A1VEELTKS8NLZB,A14R9XMZVJ6INB,A27IQHDZFQFNGG,A31QY5TASILE89,A2LWK003FFMCI5
reviewerName,Amazon Customer,amf0001,Caitlin,DebraDownSth,Diana X.
summary,Good Taste,"3.5 stars, sadly not as wonderful as I had hoped",Yum!,Unexpected flavor meld,"Not a very strong tea flavor, but still yummy ..."
unixReviewTime,1370044800,1400457600,1381190400,1369008000,1369526400


In [60]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151254 entries, 0 to 151253
Data columns (total 13 columns):
timestamp         151254 non-null datetime64[ns]
asin              151254 non-null object
helpful           151254 non-null object
overall           151254 non-null float64
reviewText        151232 non-null object
reviewTime        151254 non-null object
reviewerID        151254 non-null object
reviewerName      149761 non-null object
summary           151254 non-null object
unixReviewTime    151254 non-null int64
review_year       151254 non-null int64
review_month      151254 non-null int64
review_day        151254 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(4), object(7)
memory usage: 15.0+ MB


In [61]:
DF.to_csv("block_9_data.csv", index=False)